In [1]:
from libqubitNd import *

In [284]:
import types
def tensorate_function(old_f):
    def tensored_f(self,*args, **kwargs):
        if 'non_tensor_form' in kwargs and kwargs['non_tensor_form'] == True:
            del kwargs['non_tensor_form']
            return old_f(*args, **kwargs)
        obj = old_f(*args, **kwargs)
        try:
            if 'order' in self.__dict__ and 'identity2' in self.__dict__ and obj.type == 'oper' and len(obj.dims[0]) == 1:
                if self.order == 0:
                    return tensor(obj, self.identity2())
                else:
                    return tensor(self.identity2(), obj)
            else:
                return obj
        except AttributeError:
            return obj
    return tensored_f
            
def Tensorator(OldClass):
    class NewClass(OldClass):
        def update_or_copy(self, kvargs, params, just_update = False):
            super().update_or_copy( kvargs, params, just_update)
            if not just_update:
                for field in self.__dir__():
                    if field[0] != '_' and callable(getattr(self,field)):
                        setattr(self, field, types.MethodType(tensorate_function(getattr(self,field)), self))
                if self.identity2 == None:
                    self.identity2 = types.MethodType(OldClass.I,self)
                    
                    
    NewClass.class_params += ['order','identity2']
    NewClass.default.order = 0
    NewClass.default.identity2 = None if 'I' in dir(OldClass) else identity
    return NewClass

def Tensorator2(OldClass):
    class NewClass(OldClass):
        pass
    for field in dir(NewClass):
        if field[0] != '_' and callable(getattr(NewClass,field)):
            setattr(NewClass, field, tensorate_function(getattr(NewClass,field)))
    NewClass.class_params += ['order','identity2']
    NewClass.default.order = 0
    NewClass.default.identity2 = OldClass.I if 'I' in dir(OldClass) else identity
    return NewClass

In [285]:
TSQND = Tensorator(SingleQND)

In [286]:
a = TSQND(order = 0)
b = TSQND(order = 1)

In [298]:
class CCQ(Model):
    default = Empty()
    default.qubits = [TSQND(order = 0), TSQND(order = 1)]
    default.params = {
        'wxx' : 0.01
    }
    class_params = ['qubits', 'params']
    def H(self):
        n = [0,0]
        for i, q in enumerate(self.qubits):    
            try:
                n[i] = q.n(non_tensor_form = True)
            except AttributeError:
                n[i] = sigmax()
        return self.params['wxx']*tensor(n)        
        
    def args(self):
        
    
    def Hconst(self):
        Hq = sum([a.Hq() for a in self.qubits])
        return Hq + self.H()
    def HTD(self):
        htd = [self.H()]
        for q in self.qubits:
            htd += q.HTD()
        return htd
    

In [301]:
c = CCQ(qubits = [Original]

In [ ]:

def decorated_f(self, *args):
    obj = f(self, *args)
    if 'order' in self.__dict__ and obj.isOperator():
        if self.order == 0:
            return tensor(obj, self.identity2())
        else:
            return tensor(self.identity2(), obj)
    
        

class CCQ(Model):
    def 